In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece accelerate
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

In [ ]:
# Please specify a BASE_MODEL, e.g. BASE_MODEL = 'decapoda-research/llama-7b-hf'
BASE_MODEL = ""

# Please specify a LORA_WEIGHTS, e.g. LORA_WEIGHTS = 'mchl-labs/stambecco-7b-plus'
LORA_WEIGHTS = ""

In [ ]:
import torch
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained(BASE_MODEL)
model = LLaMAForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

model = PeftModel.from_pretrained(model, LORA_WEIGHTS, torch_dtype=torch.float16)
model.eval()

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Di seguito è riportata un'istruzione che descrive un task, insieme ad un input che fornisce un contesto più ampio. Scrivi una risposta che completi adeguatamente la richiesta.
### Istruzione:
{instruction}
### Input:
{input}
### Risposta:"""
    else:
        return f"""Di seguito è riportata un'istruzione che descrive un task. Scrivi una risposta che completi adeguatamente la richiesta.
### Istruzione:
{instruction}
### Risposta:"""

In [ ]:
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    print("Risposta:", output.split("### Risposta:")[1].strip())

In [ ]:
# Insert an instruction and an input (optional)
evaluate(input())